In [1]:
! pip install -e gym-autotrain

Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///home/jupyter-skenjeye%40broadinst-05974/AutoTrain/gym-autotrain
  Attempting uninstall: gym-autotrain
    Found existing installation: gym-autotrain 0.0.1
    Uninstalling gym-autotrain-0.0.1:
      Successfully uninstalled gym-autotrain-0.0.1
  Running setup.py develop for gym-autotrain


In [2]:
import torch
import torchvision
import torchvision.transforms as T
import torchvision.models as models

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions.laplace import Laplace
import torch.utils.data as torchdata

import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
import pickle as pkl
from pathlib import Path
from functools import partial
import pandas as pd

from tqdm.notebook import tqdm

import gym
import gym_autotrain

from gym_autotrain.envs.autotrain_env import StateLinkedList, ObservationAndState

# Data

In [3]:
DATA_ROOT = Path('./data')
DATA_SPLIT = 0.6

ENV_PATH = Path('./autotrain-run')
ENV_PATH.mkdir(exist_ok=True)

DEVICE = torch.device("cuda:3")

In [4]:

CLASSES = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

def splitds(train, test, no_signal=False, pct_cap=None):
    X = np.concatenate((train.data,test.data), axis=0)
    Y = train.targets + test.targets
    
    if pct_cap:
        cap = int(pct_cap*len(X))
        X, Y = X[:cap], Y[:cap]
        
    
    if no_signal:
        print('suffling labels')
        np.random.shuffle(Y)
    
    split_id = int(len(X) * DATA_SPLIT)
    train.data, train.targets = X[:split_id], Y[:split_id]
    test.data, test.targets = X[split_id:], Y[split_id:]

def get_dataset(tfms, no_signal=False, pct_cap=None):
    train = torchvision.datasets.CIFAR10(root=DATA_ROOT / 'cifar-10-data', train=True,
                                        download=True, transform=tfms)

    holdout = torchvision.datasets.CIFAR10(root=DATA_ROOT / 'cifar-10-data', train=False,
                                           download=True, transform=tfms)
    
    splitds(train, holdout, no_signal, pct_cap)
    
    print(f'length of trainset: [{len(train)}]; len of holdout: [{len(holdout)}]')
    
    return train, holdout


In [5]:
normalize = T.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])

TFMS = T.Compose([T.Resize(256), T.CenterCrop(224), T.ToTensor(), normalize])

train, holdout = get_dataset(TFMS,pct_cap=0.1)

Files already downloaded and verified
Files already downloaded and verified
length of trainset: [3600]; len of holdout: [2400]


In [6]:
agent = lambda dim: np.random.rand(dim)

def accuracy(data: torchdata.DataLoader, model: nn.Module): # phi
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in tqdm(data,total=len(data)):
            images, labels = batch[0].to(DEVICE), batch[1]
            outputs = model(images).cpu()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    return correct / total

# Model

In [7]:
backbone = models.resnet18(pretrained=False)
backbone.to(DEVICE)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# Env

In [9]:
env = gym.make('AutoTrain-v0')

env.init(backbone=backbone,  phi=accuracy, savedir=ENV_PATH,
         trnds=train, valds=holdout, 
         T=3, H=5, K=256, lr_init=3e-4, inter_reward=0.05,
         num_workers=4, bs=16, log_file=None)